<a href="https://colab.research.google.com/github/gwendolynstripling/datasciencegeek/blob/master/10_2_2019_DRAFT2_FeatEnG_LABS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applying Feature Engineering to BigQuery ML Models - Part 1

**Learning Objectives**

* Setup up the environbment
* Create the project dataset
* Create the feature engineering training table
* Create and evaluate the  benchmark/baseline model
* Extract numeric features
* Perform a Feature cross
* Evalute model performance

## Introduction 
In this notebook, we utilize feature engineering to improve the prediction of the fare amount for a taxi ride in New York City based on numeric, categorical, temporal, distance, and coordinate features.  We will use BigQuery ML to build a taxifare prediction model, using feature engineering to improve and create a final model.


### Set up environment variables and load necessary libraries

In [0]:
PROJECT = "XXXXX"  # Replace with your PROJECT
REGION = "XXXX"            # Choose an available region for Cloud MLE

In [0]:
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

Check that the Google BigQuery library is installed and if not, install it. 

In [0]:
!pip freeze | grep google-cloud-bigquery==1.6.1 || pip install google-cloud-bigquery==1.6.1

## The source dataset

Our dataset is hosted in [BigQuery](https://cloud.google.com/bigquery/). The taxi fare data is a publically available dataset, meaning anyone with a GCP account has access. Click [here](https://console.cloud.google.com/bigquery?project=bigquery-public-data&p=nyc-tlc&d=yellow&t=trips&page=table) to acess the dataset.


## Create the project dataset

Although the source dataset is hosted in BigQuery, in this section you create a dataset to hold your table, add data to your project, then make the data table you'll query against.  As you recall, datasets help you control access to tables and views in a project. This lab uses only one table, but you still need a dataset to hold the table.

In [0]:
# Create the feature engineering dataset ...
!bq mk feat_eng

## Verify the dataset properties

Verify that you created the dataset by viewing the dataset's properties with the bq show command.


In [0]:
#View dataset properties

%%bigquery
bq show feat_eng

## Create the training data table

Since there is already a publicly available dataset, we can simply create the training data table.  Note the WHERE clause in the below query:  This clause allows us to TRAIN a portion of the data (e.g. one million rows versus a billion rows), which keeps your query costs down. 

* Note:  The dataset in the create table code below is the one created previously, e.g. "feat_eng".  The table name is "feateng_training_data".
* **RUN** the query to create the table.

In [0]:
%%bigquery
CREATE OR REPLACE TABLE feat_eng.feateng_training_data 
AS
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  passenger_count*1.0 AS passengers,
  pickup_datetime,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat


FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 10000) = 1
 
  AND fare_amount >= 2.5
  AND passenger_count > 0
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  
 

## Verify table creation

Verify that you created the dataset by viewing the dataset's properties with the bq show command.


In [0]:
# OPTION #1 - Verify table creation

%%bigquery
bq show feat_eng.feateng_training_data 

In [0]:
# OPTION 2 - Verify table creation

%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM feat_eng.feateng_training_data 
LIMIT 0

### Create the Benchmark/Baseline Model

Next, you create a linear regression baseline model with no feature engineering.  Recall that a model in BigQuery ML represents what an ML system has learned from the training data.  A baseline model is a solution to a problem without applying any machine learning techniques.  

When creating a BQML model, you must specify the model type (in our case linear regression) and the input label (fare_amount).  Note also that we are using the training data table as the data source.

###Create the SQL statement to create the model "Benchmark Model".

In [0]:
#SQL STATEMENT HERE: 



In [0]:
#SOLUTION

%%bigquery
CREATE MODEL feat_eng.benchmark_model
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  pickup_datetime,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
  
FROM `feat_eng.feateng_training_data`


REMINDER:  The query takes several minutes to complete. After the first iteration is complete, your model (sample_model) appears in the navigation panel of the BigQuery web UI. Because the query uses a CREATE MODEL statement to create a model, you do not see query results.

You can observe the model as it's being trained by viewing the Model stats tab in the BigQuery web UI. As soon as the first iteration completes, the tab is updated. The stats continue to update as each iteration completes.



Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

### Evaluate the benchmark model
Note that BigQuery automatically split the data we gave it, and trained on only a part of the data and used the rest for evaluation.  After creating your model, you evaluate the performance of the regressor using the ML.EVALUATE function. The ML.EVALUATE function evaluates the predicted values against the actual data.

NOTE: The results are also displayed in the BigQuery GUI console under the **Evaluation** tab.

###Create the SQL statement to EVALUATE the Benchmark Model.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE THE Benchmark Model here.  
Expand the elipses below to see the solution.

In [0]:
#@title
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.benchmark_model)


###Create the SQL statement to EVALUATE the Benchmark Model using RMSE.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE THE Benchmark Model here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL feat_eng.benchmark_model)

**NOTE:** Because you performed a linear regression, the results include the following columns:

*   mean_absolute_error
*   mean_squared_error
*   mean_squared_log_error
*   median_absolute_error
*   r2_score
*   explained_variance

**Resource** for an explanation of the regression metrics:  [Regression Metrics](https://https://joshlawman.com/metrics-regression/)

**Mean squared error** (MSE) measures the difference between the values our model predicted using the test set and the actual values. You can also think of it as the distance between your regression (best fit) line and the predicted values. A smaller value is better, and our model’s .0121 MSE is very good.

**Note**:  An important metric in the evaluation results is the R2 score. The R2 score is a statistical measure that determines if the linear regression predictions approximate the actual data. 0 indicates that the model explains none of the variability of the response data around the mean. 1 indicates that the model explains all the variability of the response data around the mean.

#### EXTRACT DayOfWeek from the pickup_datetime feature.

* As you recall, DayOfWeek is an enum representing the 7 days of the week. This factory allows the enum to be obtained from the int value. The int value follows the ISO-8601 standard, from 1 (Monday) to 7 (Sunday). 

* Create a model titled "model_1" from the benchmark model and extract out the DayofWeek.


In [0]:
#Create the SQL statement to EXTRACT ouT the DayofWeek from the pickup_datetime feature.
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.model_1
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
  
FROM `feat_eng.feateng_training_data`

Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

###Create the SQL statement to EVALUATE Model_1.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE Model_1 here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.model_1)


###Create the SQL statement to EVALUATE the Model_1 using RMSE.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE Model_1 here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL feat_eng.model_1)

### EXTRACT hourofday from the pickup_datetime feature.

As you recall, **pickup_datetime** is stored as a TIMESTAMP, where the Timestamp format is retrieved in the standard output format – year-month-day hour:minute:second (e.g. 2016-01-01 23:59:59).  Hourofaday returns the integer number representing the hour number of the given date.

* Create a model titled "model_2"
* EXTRACT the hourofday from the pickup_datetime feature to improve our model's rmse.

In [0]:
#Create the SQL statement to EXTRACT HourofDay from the pickup_datetime feature.
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_2
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
  
FROM `feat_eng.feateng_training_data`

###Create the SQL statement to EVALUATE Model_2.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE Model_2 here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.model_2)


###Create the SQL statement to EVALUATE Model_2 using RMSE.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE the model here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUITON

%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL feat_eng.model_2)

### Feature cross dayofweek and hourofday 

First, let’s allow the model to learn traffic patterns by creating a new feature that combines the time of day and day of week (this is called a feature cross). 

* Modify model_2 to create a feature cross that combines the time of day and day of week.  Note:  CAST DAYOFWEEK and HOUR as strings.  Name the model "model_3".  

* In this lab, we will modify the SQL to first use the CONCAT function to concatenate (feature cross) the dayofweek and hourofday features.  Then, we will use the ML.FEATURE_CROSS, BigQuery's new pre-processing feature cross function.

Note:  BQML by default assumes that numbers are numeric features, and strings are categorical features.  We need to convert these features to strings because the Neural Network will treat 1,2,3,4,5,6,7 as numeric values.  Thus, there is no way to distinguish the time of day and day of week "numerically."


In [0]:
#Create the SQL statement to feature cross the dayofweek and hourofday using the CONCAT function.
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_3
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  #EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  #EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
  CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING), 
        CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
  
FROM `feat_eng.feateng_training_data`

### APPLY the ML.FEATURE_CROSS

BigQuery ML now has ML.FEATURE_CROSS, a pre-processing function that performs a feature cross.  

* ML.FEATURE_CROSS generates a STRUCT feature with all combinations of crossed categorical features, except for 1-degree items (the original features) and self-crossing items.  

* Syntax:  ML.FEATURE_CROSS(STRUCT(features), degree)

* The feature parameter is a categorical features separated by comma to be crossed. The maximum number of input features is 10. Unnamed feature is not allowed in features. Duplicates are not allowed in features.

* Degree(optional): The highest degree of all combinations. Degree should be in the range of [1, 4]. Default to 2.

Output: The function outputs a STRUCT of all combinations except for 1-degree items (the original features) and self-crossing items, with field names as concatenation of original feature names and values as the concatenation of the column string values.

In [0]:
#The ML.Feature_Cross statement contains errors. Modify and run the query.
%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_3
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  #EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  #EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
  #CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING), 
        #CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday,
  ML.FEATURE_CROSS(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING),
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
  
FROM `feat_eng.feateng_training_data`


In [0]:
#Solution Code

ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr


###Create the SQL statement to EVALUATE Model_3.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE Model_3 here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.model_3)


###Create the SQL statement to EVALUATE Model_3 using RMSE.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE the model here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL feat_eng.model_3)

# Applying Feature Engineering to BigQuery ML Models - PART 2

**Learning Objectives**

* Derive a coordinate features
* Feature cross coordinate features
* Code cleanup
* Apply the TRANSFORM clause
* Evalute model performance


## Introduction 
In this notebook, we continue using feature engineering to improve the prediction of the fare amount for a taxi ride in New York City. 


### Convert geolocation/coordinate features


Pickup coordinate:
*  pickup_longitude AS pickuplon
*  pickup_latitude AS pickuplat
 


Dropoff coordinate:
*   #dropoff_longitude AS dropofflon
*   #dropoff_latitude AS dropofflat

**NOTES**:
* The pick-up and drop-off longitude and latitude data are crucial to predicting the fare amount as fare amounts in NYC taxis are largely determined by the distance traveled.  Assuch, we need to  teach the model the Euclidean distance between the pick-up and drop-off points.  

* Recall that latitude and longitude allows us to specify any location on Earth using a set of coordinates.  In our training data set, we restricted our data points to only pickups and drop offs within NYC. NYC has an approximate longitude range of -74.05 to -73.75 and a latitude range of 40.63 to 40.85.

* The dataset contains information regarding the pickup and drop off coordinates. However, there is no information regarding the distance between the pickup and drop off points. Therefore, we create a new feature that calculates the distance between each pair of pickup and drop off points. We can do this using the Euclidean Distance, which is the straight-line distance between any two coordiante points.

* We need to convert those coordinates into a single column of a spatial data type.  We will use the The ST_Distance function, which returns the minimum distance between two spatial objects.  


### Derive a coordinate features

* Convert the feature coordinates into a single column of a spatial data type. Use the The ST_Distance function, which returns the minimum distance between two spatial objects.

SAMPLE CODE:
ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean




In [0]:
#SQL STATEMENT HERE:

In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_4
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  #EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  #EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
  #CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING), 
        #CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday,
  ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
  #pickuplon,
  #pickuplat,
  #dropofflon,
  #dropofflat,
  ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
  AS euclidean
  
FROM `feat_eng.feateng_training_data`

###Create the SQL statement to EVALUATE Model_4.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE Model_4 here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.model_4)


###Create the SQL statement to EVALUATE Model_4 using RMSE.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE THE Model here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL feat_eng.model_4)

### Feature cross coordinate features

In this section, we feature cross the pick-up and drop-off locations so that the model can learn pick-up-drop-off pairs that will require tolls.

This step takes the geographic point corresponding to the pickup point and grids to a 0.1-degree-latitude/longitude grid (approximately 8km x 11km in New York—we should experiment with finer resolution grids as well). Then, it concatenates the pickup and dropoff grid points to learn “corrections” beyond the Euclidean distance associated with pairs of pickup and dropoff locations.

Because the lat and lon by themselves don't have meaning, but only in conjunction, it may be useful to treat the fields as a pair instead of just using them as numeric values. However, lat and lon are continuous numbers, so we have to discretize them first. That's what SnapToGrid does. 

**REMINDER**: The ST_GEOGPOINT creates a GEOGRAPHY with a single point. ST_GEOGPOINT creates a point from the specified FLOAT64 longitude and latitude parameters and returns that point in a GEOGRAPHY value.  The ST_Distance function returns the minimum distance between two spatial objectsa.  It also returns meters for geographies and SRID units for geometrics.  


In [0]:
#The following feature cross coordinate code is incorrect.  Modify the code to feature cross the coordinate features. 

%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_4
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  #EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  #EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
  #CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING), 
        #CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday,
  ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
  #pickuplon,
  #pickuplat,
  #dropofflon,
  #dropofflat,
 ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplat,pickuplon,pickuplat), 0.05)), 
 ST_AsText(ST_GeogPoint(dropofflon, dropofflat,dropofflon), 0.04)) AS pickup_and_dropoff
  
FROM `feat_eng.feateng_training_data`


In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_5
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  #pickup_datetime,
  #EXTRACT(DAYOFWEEK FROM pickup_datetime) AS dayofweek,
  #EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
  CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING), 
        CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday,
  #pickuplon,
  #pickuplat,
  #dropofflon,
  #dropofflat,
  ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
  AS euclidean,
   CONCAT(ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01)))
  AS pickup_and_dropoff
  
FROM `feat_eng.feateng_training_data`

###Create the SQL statement to EVALUATE Model_5.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE Model_5 here.  
Expand the elipses below to see the solution.

In [0]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.model_5)


###Create the SQL statement to EVALUATE Model_5 using RMSE.  
  
Expand the elipses below to see the solution.

In [0]:
#Create the SQL statement to EVALUATE THE Model here.  
Expand the elipses below to see the solution.

In [0]:
#SOLUTION

%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL feat_eng.model_5)

### Clean up the code to see where we are.

Remove all the commented statements in the SQL statement.  We should now have a total of five input features for our model.  


1. fare_amount
2. passengers
3. day_hr
4. euclidean
5. pickup_and_dropoff

In [0]:
%%bigquery
CREATE OR REPLACE MODEL feat_eng.revised_model_6_cleanup
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['fare_amount'])  
AS
SELECT
  fare_amount,
  passengers,
  ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
  ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
  AS euclidean,
   CONCAT(ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01)))
  AS pickup_and_dropoff
  
FROM `feat_eng.feateng_training_data`

### Apply the TRANSFORM clause

Before we perform our prediction, we should encapsulate the entire feature set in a TRANSFORM clause.  BigQuery ML now supports defining data transformations during model creation, which will be automatically applied during prediction and evaluation. This is done through the TRANSFORM clause in the existing CREATE MODEL statement. By using the TRANSFORM clause, user specified transforms during training will be automatically applied during model serving (prediction, evaluation etc.) 

In our case, we are using the TRANSFORM clause to separate out the raw input data from the TRANSFORMED features.  The input columns of the TRANSFORM clause is the query_expr (AS SELECT part).  The output columns of TRANSFORM from select_list are used in training. These transformed columns are post-processed with standardization for numerics and one-hot encoding for categorical variables by default. 

The advantage of encapsulating features in the TRANSFORM is the client code doing the PREDICT doesn't change. Our model improvement is transparent to client code. Note that the TRANSFORM clause MUST be placed after the CREATE statement.


In [0]:
#Below is a model titled "final model".  There are errors in the SQL statement.
#Correct the model statements.

%%bigquery
CREATE OR REPLACE MODEL feat_eng.final_model
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') 
AS
TRANSFORM( 
  ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
      AS euclidean, 
  CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING),
      CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS hourofday, 
  CONCAT(ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01))
  ) AS pickup_and_dropoff
)
 ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr
  
OPTIONS(input_label_cols=['fare_amount'], model_type='logistic_reg') 

SELECT * FROM `feat_eng.feateng_training_data`

In [0]:
#SOLUTION
 
%%bigquery
CREATE OR REPLACE MODEL feat_eng.final_model
TRANSFORM(
  fare_amount,
  passengers, 
  ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
      AS euclidean, 
  ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
  CONCAT(ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01))
  ) AS pickup_and_dropoff
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') 
AS

SELECT * FROM `feat_eng.feateng_training_data`

###Create two SQL statements to EVALUATE the final model.  

In [0]:
#Create the SQL statement to EVALUATE final here.  
#The solution is not provided.  Refer to previous solution for syntax/example.

# Applying Feature Engineering to BigQuery ML Models - PART 3

**Learning Objectives**

* Create a predictioon model
* Evalute model performance
* Apply the bucketize function
* L2 Regularization


## Introduction 
In this notebook, we create a prediction model, evaluate the prediction model's performance, apply the bucketize function and re-evaluate the model.

### Model prediction


Now that you have evaluated your model, the next step is to use it to predict an outcome. You use your model to predict the taxifare amount.  The ML.PREDICT function is used to predict results using your model: feat_eng.final_model.  

Since this is a regression model (predicting a continuous numerical value), the best way to see how it performed is to evaluate the difference between the value predicted by the model and the ground truth benchmark score. We can do this with an ML.PREDICT query.

MODIFY **THIS INCORRECT SQL STRATEMENT"** before running the query.  Refer to the "show code" for the solution.

In [0]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL feat_eng.benchmark_model, (
    -73.982683 AS pickuplon,
    40.742104 AS pickuplat,
    -73.983766 AS dropofflon,
    40.755174 AS dropofflat,
    3.0 AS passengers,
    TIMESTAMP('2019-06-03 04:21:29.769443 UTC') AS pickup_datetime
))

In [0]:
#SOLUTION

%%bigquery
SELECT * FROM ML.PREDICT(MODEL feat_eng.final_model, (
  SELECT 
    -73.982683 AS pickuplon,
    40.742104 AS pickuplat,
    -73.983766 AS dropofflon,
    40.755174 AS dropofflat,
    3.0 AS passengers,
    TIMESTAMP('2019-06-03 04:21:29.769443 UTC') AS pickup_datetime
))

In [0]:
#Create the SQL statement to EVALUATE final model here.  
#The solution is not provided.  Refer to previous solution for syntax/example.

### L2 Regularization

Sometimes , the training RMSE is quite reasonable, but the evaluation RMSE is terrible. This is an indication of overfitting. When we do feature crosses, we run into the risk of overfitting (for example, when a particular day-hour combo doesn't have enough taxirides).

* Modify final_model below to implement L2 Regularization.



In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.final_model
TRANSFORM(
  fare_amount,
  passengers, 
  ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
      AS euclidean, 
  ML.FEATURE_CROSS(STRUCT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek,
  CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday)) AS day_hr,
  CONCAT(ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01))
  ) AS pickup_and_dropoff
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg', l2_reg=0.1) 
AS

SELECT * FROM `feat_eng.feateng_training_data`

### Apply the BUCKETIZE Function


BigQuery ML now has BUCKETIZE, a pre-processing function that creats "buckets" (bins) - e.g. it bucketizes a continuous numerical feature into a string feature with bucket names as the value.

* ML.BUCKETIZE(feature, split_points)

* feature: A numerical column.

* split_points: Array of numerical points to split the continuous values in feature into buckets. With n split points (s1, s2 … sn), there will be n+1 buckets generated. 

* Output: The function outputs a STRING for each row, which is the bucket name. bucket_name is in the format of bin_<bucket_number>, where bucket_number starts from 1.

In [0]:
#Use the ML.BUCKETIZE function to EXTRACT the HOUR FROM pickup_datetime, creating three buckets of 5, 10, 17 as hourofday..  
#CODE THE ENTIRE MODEL HERE.  Name the model "bucket_model"

In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.bucket_model
TRANSFORM(fare_amount, passengers,
          ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
              AS euclidean, 
          IF(EXTRACT(dayofweek FROM pickup_datetime) BETWEEN 2 and 6, 'weekday', 'weekend') as dayofweek,
          ML.BUCKETIZE(EXTRACT(HOUR FROM pickup_datetime), [5, 10, 17]) AS hourofday
)
OPTIONS(input_label_cols=['fare_amount'], 
        model_type='linear_reg')
AS
SELECT 
fare_amount,
passengers,
pickup_datetime

FROM  `feat_eng.feateng_training_data`

In [0]:
#Create the SQL statement to EVALUATE the model here.  
#The solution is not provided.  Refer to previous solution for syntax/example.

### L2 Regularization with BUCKETIZATION

Sometimes , the training RMSE is quite reasonable, but the evaluation RMSE is terrible. This is an indication of overfitting. When we do feature crosses, we run into the risk of overfitting (for example, when a particular day-hour combo doesn't have enough taxirides).

* Modify bucket_model to implement L2 Regularization.



In [0]:
%%bigquery
CREATE OR REPLACE MODEL feat_eng.bucket_model
TRANSFORM(*EXCEPT(pickup_datetime), fare_amount, passengers,
          ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
              AS euclidean, 
          IF(EXTRACT(dayofweek FROM pickup_datetime) BETWEEN 2 and 6, 'weekday', 'weekend') as dayofweek,
          ML.BUCKETIZE(EXTRACT(HOUR FROM pickup_datetime), [5, 10, 17]) AS hourofday
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') 

AS
SELECT 
*
FROM  `feat_eng.feateng_training_data`

In [0]:
#SOLUTION

%%bigquery
CREATE OR REPLACE MODEL feat_eng.bucket_model
TRANSFORM(*EXCEPT(pickup_datetime)
          ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) 
              AS euclidean, 
          IF(EXTRACT(dayofweek FROM pickup_datetime) BETWEEN 2 and 6, 'weekday', 'weekend') as dayofweek,
          ML.BUCKETIZE(EXTRACT(HOUR FROM pickup_datetime), [5, 10, 17]) AS hourofday
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg', l2_reg=0.1) 

AS
SELECT 
*
FROM  `feat_eng.feateng_training_data`

In [0]:
#Create the SQL statement to EVALUATE the model here.  
#The solution is not provided.  Refer to previous solution for syntax/example.

### LAK: 
Should we need to run the prediction model again - after L2 regularization -  as a best practice?

